## Práticas Atuarias
### Trabalho 1 e 2

In [1]:
# Importando bibliotecas 

import pandas as pd
import numpy as np
import os
import datetime
from dateutil.relativedelta import relativedelta

# Lendo e tratando a base de dados

In [11]:
def clean_cols(col_name):
    """Trata as colunas para ficarem padronizadas"""
    # trata casos especiais
    if col_name == 'Nº Registro':
        return 'num_registro'
    elif 'Data Nascito.' in col_name:
        suffix = col_name.split('Data Nascito.')[-1].strip()
        return 'data_nascimento_' + suffix.lower().replace(' ', '_')

    # tratamento geral
    else:
        return col_name.lower().replace(' ', '_')
    
def get_age(x, today=datetime.datetime.today):
    """Retorna a idade relativa a data passada"""
    try:
        diff = relativedelta(today(), x)
        return diff.years
    except AssertionError:
        return np.nan


# Le o arquivo original
df_original = pd.read_excel("201 Cond Trab 1 e 2 Eduardo W abr20.xls", sheet_name="Dados", header=2)
    
# Faz uma cópia e aplica a função clean_cols
df_cleanse_columns = df_original.copy()
df_cleanse_columns.columns = [clean_cols(col) for col in df_original.columns]

# faz uma cópia do DataFrame com as colunas padronizadas e transforma colunas de data em datetime objects
df = df_cleanse_columns.copy()
for i, col in enumerate(df_cleanse_columns.columns):
    if 'unnamed' in col:
        df[df_cleanse_columns.iloc[:, i-1].name] = pd.to_datetime(
            df_cleanse_columns[col].astype(str) + df_cleanse_columns.iloc[:, i-1].replace(to_replace=0, value=1).astype(str), 
            format='%Y%m', 
            errors='coerce'
        )
        df = df.drop(col, axis='columns')
        
# limpa linhas vazias
df = df.dropna(axis='index', how='all')

# adiciona a coluna de idades ao DataFrame
for i, col in enumerate(df.columns):
    if 'data_nascimento' in col:
        idades = df[col].apply(get_age)
        col_name = col.replace('data_nascimento', 'idade')
        loc = df.columns.get_loc(col)
        df.insert(loc=loc+1, column=col_name, value=idades)

df.head()

,num_registro,sexo,data_admissão,data_nascimento_titular,idade_titular,salário,data_nascimento_cônjuge,idade_cônjuge,data_nascimento_filho_a,idade_filho_a,data_nascimento_filho_b,idade_filho_b,data_nascimento_filho_c,idade_filho_c
1,1201.0,F,2016-12-01,1986-01-01,34,3529.5448,1986-01-01,34.0,2012-08-01,7.0,2019-06-01,0.0,2021-01-01,0.0
2,1202.0,M,2008-06-01,1978-04-01,42,3280.3036,1975-01-01,45.0,2014-05-01,5.0,1988-03-01,32.0,NaT,NaN
3,1203.0,M,2010-07-01,1975-09-01,44,1786.8844,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN
4,1204.0,M,2014-06-01,1972-11-01,47,1505.2966,1969-01-01,51.0,2014-09-01,5.0,2016-07-01,3.0,2005-02-01,15.0
5,1205.0,M,2005-10-01,1970-06-01,49,4601.9512,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN


Para realizar os tratamentos 